### 고객정보와 무게를 기반으로 칼로리 계산하여 출력하기 통합

In [25]:
import pandas as pd
from datetime import datetime


# 고객 정보를 CSV에서 가져오는 함수
def get_customer_info(customer_id, csv_file_path):
    try:
        # CSV 파일 읽기
        customer_data = pd.read_csv(csv_file_path, dtype={'연락처': object})
        # 고객 ID로 필터링
        customer_info = customer_data[customer_data['고객아이디'] == customer_id]
        if not customer_info.empty:
            return customer_info.to_dict(orient='records')[0]
        else:
            return None
    except FileNotFoundError:
        print(f"CSV 파일 {csv_file_path}이(가) 존재하지 않습니다.")
        return None


# BMR 계산 함수
# 미플린-지어(Mifflin-St.Jeor) 공식
def calculate_bmr(weight, height, age, sex, exercise_score):
    # 기본 BMR 계산공식
    # 기본식: (10 × 몸무게) + (6.25 × 키) – (5 × 나이)
    bmr = (10 * weight) + (6.25 * height) - (5 * age)
    # 성별에 따른 조정
    # 여성일 경우 161을 빼줌
    if sex == 'F':
        bmr -= 161
    # 남성일 경우 5를 더해줌
    elif sex == 'M':
        bmr += 5
    else:
        raise ValueError("성별은 'M' 또는 'F'로 입력해야 합니다.")

    # 운동량 기본값 설정
    if not exercise_score:  # None, '', False 모두 처리
        exercise_score = 'C'

    activity_multiplier = {
        'A': 1.9,   # 활동량 많음
        'B': 1.725,
        'C': 1.55,
        'D': 1.375,
        'E': 1.2    # 활동량 거의 없음
    }
    if exercise_score not in activity_multiplier:
        raise ValueError("운동량은 'A', 'B', 'C', 'D', 'E' 중 하나여야 합니다.")
    return bmr * activity_multiplier[exercise_score]


# 음식 칼로리 계산 함수
# food_project_food_info에 저장된 base 중량과 base 칼로리를 활용하여 칼로리 계산가능
def calculate_calories(base_weight, base_calories, consumed_weight):
    return base_calories * (consumed_weight / base_weight)


# 실시간 음식 테이블에 저장
def save_to_csv(file_path, data):
    try:
        # 기존 csv 파일 읽기
        existing_df = pd.read_csv(file_path)
        # 기존 데이터에 새로운 데이터 추가
        updated_df = pd.concat([existing_df, data], ignore_index=True)
    except FileNotFoundError:
        # 파일이 없으면 새로운 데이터로 시작
        updated_df = data
    # CSV 파일로 저장
    updated_df.to_csv(file_path, index=False)
    print(f"데이터가 {file_path}에 저장되었습니다.")


# 고객 식단 상세 정보 저장
def save_customer_diet_detail(csv_path, customer_id, total_weight, total_calories):
    today_date = datetime.now().strftime('%Y%m%d')
    try:
        # 기존 데이터 읽기
        existing_data = pd.read_csv(csv_path)
        # 같은 고객 ID와 날짜에 해당하는 데이터 필터링
        same_date_data = existing_data[
            (existing_data['customer_id'] == customer_id) &
            (existing_data['log_id'].str.startswith(f"{today_date}_{customer_id}"))
        ]
        log_number = len(same_date_data) + 1
    except FileNotFoundError:
        log_number = 1

    # 로그 아이디 생성(날짜_고객ID_번호)
    log_id = f"{today_date}_{customer_id}_{log_number}"
    # 현재시간
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 새로운 데이터 생성
    new_data = {
        "log_id": [log_id],
        "customer_id": [customer_id],
        "total_weight": [total_weight],
        "total_calories": [total_calories],
        "timestamp": [timestamp]
    }
    # 데이터를 데이터프레임으로 변환 후 저장
    new_df = pd.DataFrame(new_data)
    save_to_csv(csv_path, new_df)


# 음식 데이터 처리
def process_food_data(plate_food_data, csv_food_path, csv_customer_detail_path, customer_id):
    food_data_path = 'FOOD_DB/food_project_food_info.csv'
    food_data = pd.read_csv(food_data_path)

    total_calories = 0  # 총 칼로리 초기화
    total_weight = 0    # 총 섭취량 초기화
    real_time_food_info = []    # 실시간 음식정보를 저장할 리스트

    for food_id, measured_weight, measured_volume in plate_food_data:
        # 음식 정보 가져오기
        food_info = food_data[food_data['음식 아이디'] == food_id]
        # 딕셔너리로 변환하기
        food_info_dict = food_info.to_dict(orient='records')[0]
        if not food_info.empty:
            # 데이터프레임에서 값을 가져올때
            # name = food_info['음식명'].values[0]
            # base_weight = food_info['중량(g)'].values[0]
            # base_calories = food_info['에너지(kcal)'].values[0]

            # 데이터프레임을 딕셔너리로 변환한 후에는 키값만 활용하면 값을 가져올 수 있음
            name = food_info_dict['음식명']
            base_weight = food_info_dict['중량(g)']
            base_calories = food_info_dict['에너지(kcal)']

            # 칼로리 계산
            calculated_calories = calculate_calories(base_weight, base_calories, measured_weight)

            # 실시간 음식 정보 추가
            real_time_food_info.append({
                "food_id": food_id,
                "name": name,
                "volume": measured_volume,  # 부피는 태윤이가 정수형으로만 받는다고해서 그냥받음
                "weight": round(measured_weight, 2),    # 무게는 소수점 2자리
                "calories": round(calculated_calories, 2),  # 칼로리도 소수점 2자리
                "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

            # 총 칼로리와 총 섭취량 누적
            total_calories = round(total_calories + calculated_calories, 2)
            total_weight = round(total_weight + measured_weight, 2)
        else:
            print(f"food_id {food_id}에 해당하는 음식 정보를 찾을 수 없습니다.")

    if real_time_food_info:
        # 실시간 음식 정보를 저장
        save_to_csv(csv_food_path, pd.DataFrame(real_time_food_info))
    # 고객 식단 상세 정보 저장
    save_customer_diet_detail(csv_customer_detail_path, customer_id, total_weight, total_calories)

    # 총 섭취량 및 칼로리 출력
    print(f"총 누적량: {total_weight:.2f}g")
    print(f"총 칼로리: {total_calories:.2f} kcal")


# 실행
user_csv_file_path = 'FOOD_DB/food_project_user_info.csv'
real_time_csv_path = 'FOOD_DB/real_time_food_info.csv'
customer_diet_csv_path = 'FOOD_DB/customer_diet_detail.csv'

# 고객 ID 입력
customer_id = input("고객 ID를 입력하세요: ")
customer_info = get_customer_info(customer_id, user_csv_file_path)

if customer_info:
    try:
        birth_date = str(customer_info['생년월일'])
        weight = customer_info['몸무게']
        height = customer_info['키']
        sex = customer_info['성별']
        exercise_score = customer_info['운동량']
        name = customer_info['고객명']

        # 나이 계산
        today = datetime.now()
        birth_year, birth_month, birth_day = int(birth_date[:4]), int(birth_date[4:6]), int(birth_date[6:])
        age = today.year - birth_year - ((today.month, today.day) < (birth_month, birth_day))

        # BMR 및 한 끼 권장 식사량 계산
        bmr_value = calculate_bmr(weight, height, age, sex, exercise_score)
        one_meal_value = bmr_value / 3

        # 고객 정보 출력
        print(f"고객명: {name}")
        print(f"나이: {age}세")
        print(f"BMR: {bmr_value:.2f} kcal")
        print(f"한 끼 권장 식사량: {one_meal_value:.2f} kcal")

        # 음식 데이터 입력
        plate_food_data = []
        while True:
            user_input = input("음식 데이터 (food_id, weight, volume 입력 또는 'end'로 종료): ")
            if user_input.lower() == 'end':
                print("배식완료.")
                break
            try:
                food_id, weight, volume = map(str.strip, user_input.split(","))
                plate_food_data.append((int(food_id), float(weight), float(volume)))
            except ValueError:
                print("잘못된 입력 형식입니다. 'food_id,weight,volume' 형식으로 입력하세요.")

        if plate_food_data:
            process_food_data(plate_food_data, real_time_csv_path, customer_diet_csv_path, customer_id)
        else:
            print("입력된 음식 데이터가 없습니다. 종료합니다.")

    except Exception as e:
        print(f"오류 발생: {e}")
else:
    print("해당하는 고객 ID를 찾을 수 없습니다.")


고객명: 홍유경
나이: 99세
BMR: 1089.30 kcal
한 끼 권장 식사량: 363.10 kcal
배식완료.
데이터가 FOOD_DB/real_time_food_info.csv에 저장되었습니다.
데이터가 FOOD_DB/customer_diet_detail.csv에 저장되었습니다.
총 누적량: 100.00g
총 칼로리: 36.77 kcal


### 위 코드에 양추정 코드(실시간 음식정보에 추가예정)도 생성 추가하기 

In [28]:
import pandas as pd

# 파일 경로
csv_path = 'FOOD_DB/quantity_min_max.csv'

try:
    # CSV 파일 읽기
    data = pd.read_csv(csv_path)
    
    # 데이터 타입 출력
    print("각 컬럼의 데이터 타입:")
    print(data.dtypes)
    
    # 데이터 샘플 출력 (첫 5행)
    print("\nCSV 데이터 샘플:")
    print(data.head())

except FileNotFoundError:
    print(f"파일 {csv_path}을(를) 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

각 컬럼의 데이터 타입:
음식아이디     int64
음식명      object
최소량       int64
최대량       int64
dtype: object

CSV 데이터 샘플:
   음식아이디    음식명  최소량  최대량
0      0     쌀밥   50  450
1      1   된장찌개  100  600
2      2    떡갈비   20  180
3      3   달걀말이   30  250
4      4  시금치나물   15  120


In [45]:
import pandas as pd
import numpy as np


# 양추정 기준 데이터 불러오기
def load_min_max_table(file_path):
    try:
        reference_table = pd.read_csv(file_path, encoding='utf-8')
        return reference_table
    except FileNotFoundError:
        raise FileNotFoundError(f"CSV 파일 {file_path}을(를) 찾을 수 없습니다.")
    
# 특정 음식의 양 추정 코드 구간 설정
def calculate_q_ranges(food_id, min_max_table):
    # 음식 ID로 데이터 필터링
    food_info = min_max_table[min_max_table['음식아이디'] == food_id]
    if food_info.empty:
        raise ValueError(f"음식 ID {food_id}에 대한 정보를 찾을 수 없습니다.")
    
    # 최소량과 최대량 확인
    min_quantity = food_info['최소량'].values[0]
    max_quantity = food_info['최대량'].values[0]
    
    # Q1~Q5 구간 계산
    quantities = np.linspace(min_quantity, max_quantity, 5)
    ranges = {f"Q{i+1}": quantities[i] for i in range(len(quantities))}
    return ranges


# 양 추정 코드 정의(Q1-Q5)
def determine_q_category(measured_weight, q_ranges):
    q_values = list(q_ranges.values())
    for i in range(len(q_values) - 1):
        if q_values[i] <= measured_weight <= q_values[i + 1]:
            return f"Q{i+1}"
    return "Q5" if measured_weight > q_values[-1] else "Q1"


# 저장 
def save_to_csv(food_id, measured_weight, category, csv_path):
    new_entry = {
        'food_id': food_id,
        'measured_weight': measured_weight,
        'category': category
    }
    
    try:
        # 기존 데이터 로드
        existing_data = pd.read_csv(csv_path)
        updated_data = pd.concat([existing_data, pd.DataFrame([new_entry])], ignore_index=True)
    except FileNotFoundError:
        # 파일이 없으면 새로 생성
        updated_data = pd.DataFrame([new_entry])
    
    # CSV 파일 저장
    updated_data.to_csv(csv_path, index=False)
    print(f"결과가 {csv_path}에 저장되었습니다.")

# 실행
q_min_max_path = 'FOOD_DB/quantity_min_max.csv'  # 양추정 기준 CSV 파일 경로
# 우선은 별도의 파일을 생성하여 저장하는 테스트를 해보고 추후에는 실시간 음식정보 테이블에 저장하는 것으로 수정 예정
#(고객/칼로리 코드와 통합 후)
output_csv_path = 'FOOD_DB/quantity_min_max_output.csv'  # 저장할 CSV 파일 경로

try:
    # 양추정 기준 테이블 로드
    min_max_table = load_min_max_table(q_min_max_path)
except FileNotFoundError as e:
    print(e)
    exit()

# 음식 ID 및 측정된 무게 입력
food_id = int(input("음식 ID를 입력하세요: "))
measured_weight = float(input("측정된 무게를 입력하세요 (g): "))

try:
    # Q1~Q5 구간 계산
    q_ranges = calculate_q_ranges(food_id, min_max_table)
    print("Q1~Q5 구간:", q_ranges)

    # 범주 확인
    category = determine_q_category(measured_weight, q_ranges)
    print(f"측정된 무게 {measured_weight}g은 {category}에 속합니다.")

    # 결과 저장
    save_to_csv(food_id, measured_weight, category, output_csv_path)

except ValueError as e:
    print(f"오류 발생: {e}")


Q1~Q5 구간: {'Q1': 100.0, 'Q2': 225.0, 'Q3': 350.0, 'Q4': 475.0, 'Q5': 600.0}
측정된 무게 150.0g은 Q1에 속합니다.
결과가 FOOD_DB/quantity_min_max_output.csv에 저장되었습니다.
